In [1]:
import json
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.special import softmax as scipy_softmax

In [3]:
df = pd.read_csv("./processed_data_course_based.csv")

In [5]:
df

,Student Number,Course Title,Course Credit,Grades,Course Semester,GPA,Completed Credits,Semester GPA,Semester Credit,Grade 1 Rate,Grade 2 Rate,Grade 3 Rate,Grade 4 Rate,Grade 5 Rate,Mean Grade,STDEV Grade,Mean GPA,STDEV GPA,Department Code
0,0,Ideological and Moral Cultivation and Legal Fo...,2.5,4,1,3.102564,19.5,3.102564,19.5,0.000000,0.023560,0.486911,0.408377,0.081152,3.547120,0.677055,3.307021,0.516455,BLG
1,0,Success: Career Planning,1.0,3,1,3.102564,19.5,3.102564,19.5,0.005236,0.086387,0.557592,0.052356,0.298429,3.552356,1.025211,3.307021,0.516455,BLG
2,0,Introduction to Computer Science,2.0,1,1,3.102564,19.5,3.102564,19.5,0.007853,0.107330,0.358639,0.429319,0.096859,3.500000,0.841040,3.307021,0.516455,BLG
3,0,Advanced Mathematics A(1),5.5,4,1,3.102564,19.5,3.102564,19.5,0.143979,0.193717,0.204188,0.240838,0.217277,3.193717,1.359003,3.307021,0.516455,BLG
4,0,College English A(1),4.0,3,1,3.102564,19.5,3.102564,19.5,0.028796,0.256545,0.439791,0.253927,0.020942,2.981675,0.843178,3.307021,0.516455,BLG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095,381,J2EE Framework,4.5,3,6,2.790076,131.0,2.800000,20.0,0.007853,0.120419,0.348168,0.400524,0.123037,3.510471,0.886555,3.220775,0.515869,BLG
19096,381,Intellectual Property and Software Protection,1.0,4,7,2.838129,139.0,3.625000,8.0,0.000000,0.151832,0.227749,0.366492,0.253927,3.722513,1.007305,3.241516,0.494006,BLG
19097,381,Human-Computer Interaction Technology,2.0,5,7,2.838129,139.0,3.625000,8.0,0.000000,0.060209,0.413613,0.408377,0.117801,3.583770,0.774968,3.241516,0.494006,BLG
19098,381,Software Development and Testing Training,3.0,3,7,2.838129,139.0,3.625000,8.0,0.000000,0.018325,0.476440,0.463351,0.041885,3.528796,0.608736,3.241516,0.494006,BLG


In [7]:
df.drop([df.columns[0], df.columns[1]] ,inplace=True, axis=1)

In [9]:
df["Course Level"] = "Undergraduate"
df['Status'] = np.where(df['Grades'] != 1, 'Successful', 'Unsuccessful')
conditions = [
    (df['Course Semester'] <= 2),
    (df['Course Semester'] >= 3) & (df['Course Semester'] <= 4),
    (df['Course Semester'] >= 5) & (df['Course Semester'] <= 6),
    (df['Course Semester'] == 7)
]

choices = ['Freshman', 'Sophomore', 'Junior', 'Senior']

# Create the new 'standing' column
df['Standing'] = np.select(conditions, choices, default=None)

# Create Course Year column
choices = [1, 2, 3, 4]

df["Course Year"] = np.select(conditions, choices, default=None)
df

,Course Credit,Grades,Course Semester,GPA,Completed Credits,Semester GPA,Semester Credit,Grade 1 Rate,Grade 2 Rate,Grade 3 Rate,...,Grade 5 Rate,Mean Grade,STDEV Grade,Mean GPA,STDEV GPA,Department Code,Course Level,Status,Standing,Course Year
0,2.5,4,1,3.102564,19.5,3.102564,19.5,0.000000,0.023560,0.486911,...,0.081152,3.547120,0.677055,3.307021,0.516455,BLG,Undergraduate,Successful,Freshman,1
1,1.0,3,1,3.102564,19.5,3.102564,19.5,0.005236,0.086387,0.557592,...,0.298429,3.552356,1.025211,3.307021,0.516455,BLG,Undergraduate,Successful,Freshman,1
2,2.0,1,1,3.102564,19.5,3.102564,19.5,0.007853,0.107330,0.358639,...,0.096859,3.500000,0.841040,3.307021,0.516455,BLG,Undergraduate,Unsuccessful,Freshman,1
3,5.5,4,1,3.102564,19.5,3.102564,19.5,0.143979,0.193717,0.204188,...,0.217277,3.193717,1.359003,3.307021,0.516455,BLG,Undergraduate,Successful,Freshman,1
4,4.0,3,1,3.102564,19.5,3.102564,19.5,0.028796,0.256545,0.439791,...,0.020942,2.981675,0.843178,3.307021,0.516455,BLG,Undergraduate,Successful,Freshman,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095,4.5,3,6,2.790076,131.0,2.800000,20.0,0.007853,0.120419,0.348168,...,0.123037,3.510471,0.886555,3.220775,0.515869,BLG,Undergraduate,Successful,Junior,3
19096,1.0,4,7,2.838129,139.0,3.625000,8.0,0.000000,0.151832,0.227749,...,0.253927,3.722513,1.007305,3.241516,0.494006,BLG,Undergraduate,Successful,Senior,4
19097,2.0,5,7,2.838129,139.0,3.625000,8.0,0.000000,0.060209,0.413613,...,0.117801,3.583770,0.774968,3.241516,0.494006,BLG,Undergraduate,Successful,Senior,4
19098,3.0,3,7,2.838129,139.0,3.625000,8.0,0.000000,0.018325,0.476440,...,0.041885,3.528796,0.608736,3.241516,0.494006,BLG,Undergraduate,Successful,Senior,4


In [11]:
df = pd.concat([df, pd.get_dummies(df['Course Year'], prefix='Course Year'), pd.get_dummies(df['Department Code'], prefix='Department Code'), pd.get_dummies(df['Course Level'], prefix='Course Level'), pd.get_dummies(df['Standing'], prefix='Standing'), pd.get_dummies(df['Status'], prefix='Status')], axis=1)
df.drop(['Course Year', 'Department Code', 'Course Level', 'Status', 'Standing'], axis=1, inplace=True)

In [13]:
columns = df.columns

In [15]:
le = LabelEncoder()
le.fit(['1', '2', '3', '4', '5'])

LabelEncoder()

In [17]:
def standardize(X_train, X_test):
    X_train_cols = X_train.columns
    X_test_cols = X_test.columns
    sc = StandardScaler()
    fitted_sc = sc.fit(X_train)
    X_train_std = pd.DataFrame(fitted_sc.transform(X_train), columns=X_train_cols)
    X_test_std = pd.DataFrame(fitted_sc.transform(X_test), columns=X_test_cols)
    return X_train_std, X_test_std

In [19]:
df

,Course Credit,Grades,Course Semester,GPA,Completed Credits,Semester GPA,Semester Credit,Grade 1 Rate,Grade 2 Rate,Grade 3 Rate,...,Course Year_3,Course Year_4,Department Code_BLG,Course Level_Undergraduate,Standing_Freshman,Standing_Junior,Standing_Senior,Standing_Sophomore,Status_Successful,Status_Unsuccessful
0,2.5,4,1,3.102564,19.5,3.102564,19.5,0.000000,0.023560,0.486911,...,False,False,True,True,True,False,False,False,True,False
1,1.0,3,1,3.102564,19.5,3.102564,19.5,0.005236,0.086387,0.557592,...,False,False,True,True,True,False,False,False,True,False
2,2.0,1,1,3.102564,19.5,3.102564,19.5,0.007853,0.107330,0.358639,...,False,False,True,True,True,False,False,False,False,True
3,5.5,4,1,3.102564,19.5,3.102564,19.5,0.143979,0.193717,0.204188,...,False,False,True,True,True,False,False,False,True,False
4,4.0,3,1,3.102564,19.5,3.102564,19.5,0.028796,0.256545,0.439791,...,False,False,True,True,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19095,4.5,3,6,2.790076,131.0,2.800000,20.0,0.007853,0.120419,0.348168,...,True,False,True,True,False,True,False,False,True,False
19096,1.0,4,7,2.838129,139.0,3.625000,8.0,0.000000,0.151832,0.227749,...,False,True,True,True,False,False,True,False,True,False
19097,2.0,5,7,2.838129,139.0,3.625000,8.0,0.000000,0.060209,0.413613,...,False,True,True,True,False,False,True,False,True,False
19098,3.0,3,7,2.838129,139.0,3.625000,8.0,0.000000,0.018325,0.476440,...,False,True,True,True,False,False,True,False,True,False


In [21]:
def get_train_data(df, train_sem, columns):
    dataFrame = pd.DataFrame(columns=columns)
    for sem in train_sem:
        dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
    
    X_train = dataFrame.drop('Course Semester', axis=1)
    y_train = le.transform(X_train.pop('Grades'))
    return X_train, y_train

In [23]:
def define_structure(X, Y):
    input_unit = X.shape[0] # size of input layer
    hidden_unit = X.shape[0] # hidden layer of size 4
    output_unit = Y.shape[0] # size of output layer
    return (input_unit, hidden_unit, output_unit)

In [25]:
def parameters_initialization(input_unit, hidden_unit, output_unit):
    np.random.seed(41)
    W1 = np.random.randn(hidden_unit, input_unit) * 0.01
    b1 = np.zeros((hidden_unit, 1))
    W2 = np.random.randn(output_unit, hidden_unit) * 0.01
    b2 = np.zeros((output_unit, 1))
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [27]:
def softmax(x):
    return scipy_softmax(x)

In [29]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2)
    cache = {"Z1": Z1,"A1": A1,"Z2": Z2,"A2": A2}
    
    return A2, cache

In [31]:
def entropy_cost(A2, Y, parameters):
    #number of training example
    m = Y.shape[1]
    logprobs = np.multiply(np.log(A2), Y)
    cost = - np.sum(logprobs) / m
    cost = float(np.squeeze(cost))
                                    
    return cost

In [33]:
def backward_propagation(parameters, cache, X, Y):
    #number of training example
    m = X.shape[1]
    
    W1 = parameters['W1']
    W2 = parameters['W2']
    A1 = cache['A1']
    A2 = cache['A2']
   
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    
    grads = {"dW1": dW1, "db1": db1, "dW2": dW2,"db2": db2}
    
    return grads

In [35]:
def gradient_descent(parameters, grads, learning_rate = 0.01):
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
   
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1, "b1": b1,"W2": W2,"b2": b2}
    
    return parameters

In [37]:
def neural_network_model(X, Y, hidden_unit, num_iterations = 98):
    np.random.seed(3)
    input_unit = define_structure(X, Y)[0]
    output_unit = define_structure(X, Y)[2]
    
    parameters = parameters_initialization(input_unit, hidden_unit, output_unit)
   
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    for i in range(0, num_iterations):
        A2, cache = forward_propagation(X, parameters)
        cost = entropy_cost(A2, Y, parameters)
        grads = backward_propagation(parameters, cache, X, Y)
        parameters = gradient_descent(parameters, grads)
        if i % 5 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
    return parameters

In [39]:
def prediction(parameters, X):
    A2, cache = forward_propagation(X, parameters)
    predictions = np.round(A2)
    
    return predictions

In [41]:
def get_error_score(df, columns):
    error_scores = {}
    sorted_semesters = sorted(set(df.iloc[:, 2]))
    for sem_idx in range(1, len(sorted_semesters)):
        training_sem = sorted_semesters[:sem_idx]
        test_sem = sorted_semesters[sem_idx]
        X_train, y_train = get_train_data(df, training_sem, columns)
        X_test = df[df.iloc[:, 2] == test_sem]
        X_test.drop('Course Semester', axis=1, inplace=True)
        y_test = le.transform(X_test.pop('Grades'))
        
        X_train, X_test = standardize(X_train, X_test)
        
        X_train = X_train.T.to_numpy()   # (number of attributes, number of samples)
        y_train = y_train.reshape(1, y_train.shape[0])   # (1, number of samples)
        
        X_test = X_test.T.to_numpy()   # (number of attributes, number of samples)
        y_test = y_test.reshape(1, y_test.shape[0])   # (1, number of samples)
        
        parameters = neural_network_model(X_train, y_train, 58, num_iterations=490)
        
        y_pred_test = prediction(parameters, X_test)
        rmse_test = round(np.sqrt(mean_squared_error(y_test[0], y_pred_test[0])), 3)
        mae_test = round(mean_absolute_error(y_test[0], y_pred_test[0]), 3)
        
        y_pred_train = prediction(parameters, X_train)
        rmse_train = round(np.sqrt(mean_squared_error(y_train[0], y_pred_train[0])), 3)
        mae_train = round(mean_absolute_error(y_train[0], y_pred_train[0]),3)
        
        error_scores.setdefault(sem_idx, {})
        error_scores[sem_idx]['rmse'] = [rmse_train, rmse_test]
        error_scores[sem_idx]['mae'] = [mae_train, mae_test]        
        
    return error_scores

In [43]:
model_results = {}

In [45]:
scores = get_error_score(df, columns)
model_results['NN'] = scores

C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\982985509.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\393110090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop('Course Semester', axis=1, inplace=True)


Cost after iteration 0: 18.911436
Cost after iteration 5: 18.911047
Cost after iteration 10: 18.910634
Cost after iteration 15: 18.910182
Cost after iteration 20: 18.909672
Cost after iteration 25: 18.909084
Cost after iteration 30: 18.908395
Cost after iteration 35: 18.907577
Cost after iteration 40: 18.906597
Cost after iteration 45: 18.905415
Cost after iteration 50: 18.903984
Cost after iteration 55: 18.902248
Cost after iteration 60: 18.900141
Cost after iteration 65: 18.897588
Cost after iteration 70: 18.894509
Cost after iteration 75: 18.890817
Cost after iteration 80: 18.886438
Cost after iteration 85: 18.881316
Cost after iteration 90: 18.875451
Cost after iteration 95: 18.868936
Cost after iteration 100: 18.862028
Cost after iteration 105: 18.855240
Cost after iteration 110: 18.849470
Cost after iteration 115: 18.846173
Cost after iteration 120: 18.847557
Cost after iteration 125: 18.856825
Cost after iteration 130: 18.878407
Cost after iteration 135: 18.918182
Cost after ite

C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\3765939947.py:4: RuntimeWarning: divide by zero encountered in log
  logprobs = np.multiply(np.log(A2), Y)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\3765939947.py:4: RuntimeWarning: invalid value encountered in multiply
  logprobs = np.multiply(np.log(A2), Y)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\982985509.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\393110090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

Cost after iteration 0: 19.788679
Cost after iteration 5: 19.788071
Cost after iteration 10: 19.787433
Cost after iteration 15: 19.786736
Cost after iteration 20: 19.785950
Cost after iteration 25: 19.785043
Cost after iteration 30: 19.783975
Cost after iteration 35: 19.782699
Cost after iteration 40: 19.781160
Cost after iteration 45: 19.779293
Cost after iteration 50: 19.777019
Cost after iteration 55: 19.774248
Cost after iteration 60: 19.770874
Cost after iteration 65: 19.766782
Cost after iteration 70: 19.761852
Cost after iteration 75: 19.755974
Cost after iteration 80: 19.749066
Cost after iteration 85: 19.741119
Cost after iteration 90: 19.732258
Cost after iteration 95: 19.722839
Cost after iteration 100: 19.713586
Cost after iteration 105: 19.705784
Cost after iteration 110: 19.701532
Cost after iteration 115: 19.704032
Cost after iteration 120: 19.717921
Cost after iteration 125: 19.749557
Cost after iteration 130: 19.807207
Cost after iteration 135: 19.901000
Cost after ite

C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\982985509.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\393110090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop('Course Semester', axis=1, inplace=True)


...

Cost after iteration 0: 20.520748
Cost after iteration 5: 20.519934
Cost after iteration 10: 20.519072
Cost after iteration 15: 20.518123
Cost after iteration 20: 20.517047
Cost after iteration 25: 20.515796
Cost after iteration 30: 20.514314
Cost after iteration 35: 20.512537
Cost after iteration 40: 20.510386
Cost after iteration 45: 20.507769
Cost after iteration 50: 20.504577
Cost after iteration 55: 20.500682
Cost after iteration 60: 20.495943
Cost after iteration 65: 20.490204
Cost after iteration 70: 20.483312
Cost after iteration 75: 20.475134
Cost after iteration 80: 20.465598
Cost after iteration 85: 20.454760
Cost after iteration 90: 20.442895
Cost after iteration 95: 20.430655
Cost after iteration 100: 20.419268
Cost after iteration 105: 20.410814
Cost after iteration 110: 20.408560
Cost after iteration 115: 20.417319
Cost after iteration 120: 20.443789
Cost after iteration 125: 20.496756
Cost after iteration 130: 20.587047
Cost after iteration 135: 20.727080
Cost after ite

C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\982985509.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\393110090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop('Course Semester', axis=1, inplace=True)


Cost after iteration 0: 21.385400
Cost after iteration 5: 21.384537
Cost after iteration 10: 21.383620
Cost after iteration 15: 21.382606
Cost after iteration 20: 21.381450
Cost after iteration 25: 21.380098
Cost after iteration 30: 21.378487
Cost after iteration 35: 21.376543
Cost after iteration 40: 21.374178
Cost after iteration 45: 21.371286
Cost after iteration 50: 21.367743
Cost after iteration 55: 21.363405
Cost after iteration 60: 21.358114
Cost after iteration 65: 21.351703
Cost after iteration 70: 21.344015
Cost after iteration 75: 21.334937
Cost after iteration 80: 21.324459
Cost after iteration 85: 21.312762
Cost after iteration 90: 21.300371
Cost after iteration 95: 21.288358
Cost after iteration 100: 21.278635
Cost after iteration 105: 21.274328
Cost after iteration 110: 21.280205
Cost after iteration 115: 21.303109
Cost after iteration 120: 21.352282
Cost after iteration 125: 21.439422
Cost after iteration 130: 21.578288
Cost after iteration 135: 21.783708
Cost after ite

C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\982985509.py:4: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dataFrame = pd.concat([dataFrame, df[df.iloc[:, 2] == sem]], ignore_index=True)
C:\Users\casperrrr\AppData\Local\Temp\ipykernel_28972\393110090.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop('Course Semester', axis=1, inplace=True)


Cost after iteration 0: 22.158871
Cost after iteration 5: 22.158076
Cost after iteration 10: 22.157231
Cost after iteration 15: 22.156298
Cost after iteration 20: 22.155234
Cost after iteration 25: 22.153991
Cost after iteration 30: 22.152513
Cost after iteration 35: 22.150732
Cost after iteration 40: 22.148568
Cost after iteration 45: 22.145926
Cost after iteration 50: 22.142697
Cost after iteration 55: 22.138752
Cost after iteration 60: 22.133955
Cost after iteration 65: 22.128162
Cost after iteration 70: 22.121246
Cost after iteration 75: 22.113130
Cost after iteration 80: 22.103842
Cost after iteration 85: 22.093608
Cost after iteration 90: 22.082994
Cost after iteration 95: 22.073111
Cost after iteration 100: 22.065890
Cost after iteration 105: 22.064440
Cost after iteration 110: 22.073444
Cost after iteration 115: 22.099548
Cost after iteration 120: 22.151624
Cost after iteration 125: 22.240756
Cost after iteration 130: 22.379801
Cost after iteration 135: 22.582420
Cost after ite

In [47]:
with open('new_nn_results.json', 'w') as fw:
    json.dump(model_results, fw)